In [1]:
!pip install transformers[torch] datasets evaluate accelerate pandas scikit-learn

import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 31.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 7.3 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 89.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 49.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/19 [regex]
    Found existing installation: pyarrow 20.0.0━━━━━━━━━━━━━━━  2/19 [regex]
    Uninstalling pyarrow-20.0.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/19 [regex]
      Successfully uninstalled pyarrow-20.0.0━━━━━━━━━━━━━━━━━━━━━  3/19 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [evaluate]/19 [evaluate]ers]ub]


In [2]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    # 사용 가능한 GPU 이름 확인
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU를 사용하고 있습니다. 할당된 GPU: {gpu_name}")
else:
    print("GPU를 사용하고 있지 않습니다. 런타임 유형을 확인해주세요.")

GPU를 사용하고 있습니다. 할당된 GPU: Tesla T4


In [3]:
# ==============================================================================
# STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
# ==============================================================================

try:
    train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
    test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
    print("파일 로딩 성공!")
except FileNotFoundError:
    print("오류: 'ratings_train.txt' 또는 'ratings_test.txt' 파일을 찾을 수 없습니다.")
    print("왼쪽 파일 메뉴에 데이터 파일이 업로드되었는지 확인해주세요.")
    # 파일이 없는 경우, 코드를 중단합니다.
    exit()

# 결측치 (NaN)가 포함된 행 제거
train_df = train_df.dropna()
test_df = test_df.dropna()
print(f"결측치 제거 후 훈련 데이터 샘플 수: {len(train_df)}")
print(f"결측치 제거 후 테스트 데이터 샘플 수: {len(test_df)}")

# Pandas DataFrame을 Hugging Face Dataset 객체로 변환
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# DatasetDict으로 묶어서 관리
nsmc_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print("\nHugging Face Dataset 구성 완료:")
print(nsmc_dataset)

파일 로딩 성공!
결측치 제거 후 훈련 데이터 샘플 수: 149995
결측치 제거 후 테스트 데이터 샘플 수: 49997

Hugging Face Dataset 구성 완료:
DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label', '__index_level_0__'],
        num_rows: 149995
    })
    test: Dataset({
        features: ['id', 'document', 'label', '__index_level_0__'],
        num_rows: 49997
    })
})


In [4]:
# ==============================================================================
# STEP 2. klue/bert-base 모델 및 토크나이저 불러오기
# ==============================================================================
MODEL_NAME = "klue/bert-base"

# AutoTokenizer를 사용하여 모델에 맞는 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# AutoModelForSequenceClassification을 사용하여 분류 모델 불러오기
# 라벨은 긍정(1), 부정(0) 두 가지이므로 num_labels=2로 설정
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

print(f"'{MODEL_NAME}' 모델 및 토크나이저 로딩 완료.")

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'klue/bert-base' 모델 및 토크나이저 로딩 완료.


In [5]:
# ==============================================================================
# STEP 3. 데이터셋 전처리
# ==============================================================================

def preprocess_function(examples):
    """'document' 컬럼의 텍스트를 토크나이저로 처리하는 함수"""
    return tokenizer(examples['document'], truncation=True, padding='max_length', max_length=128)

# map 함수를 사용하여 전체 데이터셋에 전처리 함수를 일괄 적용 (batched=True로 속도 향상)
tokenized_nsmc = nsmc_dataset.map(preprocess_function, batched=True)

# 불필요한 컬럼 제거 및 포맷 설정
tokenized_nsmc = tokenized_nsmc.remove_columns(['id', 'document', '__index_level_0__'])
tokenized_nsmc.set_format("torch")


print("데이터셋 토큰화 및 전처리 완료:")
print(tokenized_nsmc['train'][0])

Map:   0%|          | 0/149995 [00:00<?, ? examples/s]

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

데이터셋 토큰화 및 전처리 완료:
{'label': tensor(0), 'input_ids': tensor([   2, 1376,  831, 2604,   18,   18, 4229, 9801, 2075, 2203, 2182, 4243,
           3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
# ==============================================================================
# STEP 4. Fine-tuning을 통한 모델 성능 향상 (Accuracy > 90%)
# ==============================================================================

# 평가 지표 설정 (정확도)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """예측 결과와 실제 라벨을 비교하여 정확도를 계산하는 함수"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# TrainingArguments 설정
training_args_step4 = TrainingArguments(
    output_dir="./results_step4",
    eval_strategy="epoch",                 # evaluation_strategy -> eval_strategy
    save_strategy="epoch",                 # save_strategy -> save_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [7]:
# Trainer 객체 생성
trainer_step4 = Trainer(
    model=model,
    args=training_args_step4,
    train_dataset=tokenized_nsmc["train"],
    eval_dataset=tokenized_nsmc["test"],
    compute_metrics=compute_metrics,
)

# 모델 학습 시작
start_time_step4 = time.time()
trainer_step4.train()
end_time_step4 = time.time()
training_time_step4 = end_time_step4 - start_time_step4

# 모델 성능 평가
eval_metrics_step4 = trainer_step4.evaluate()
accuracy_step4 = eval_metrics_step4['eval_accuracy']

Epoch,Training Loss,Validation Loss,Accuracy
1,0.252200,0.236414,0.902434
2,0.181900,0.243430,0.905314
3,0.132900,0.265184,0.906814


In [8]:
print(f"훈련 시간: {training_time_step4:.2f} 초")
print(f"평가 정확도: {accuracy_step4:.4f}")
if accuracy_step4 > 0.90:
    print("목표 정확도 90% 달성")
else:
    print("목표 정확도 90% 달성 실패")

훈련 시간: 9843.84 초
평가 정확도: 0.9068
목표 정확도 90% 달성


In [9]:
# ==============================================================================
# STEP 5. Bucketing 적용 및 결과 비교
# ==============================================================================

# 새로운 학습을 위해 모델을 초기 상태로 다시 불러옵니다.
model_bucket = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Bucketing을 적용한 TrainingArguments
training_args_step5 = TrainingArguments(
    output_dir="./results_step5_bucketing",
    eval_strategy="epoch",                 # evaluation_strategy -> eval_strategy
    save_strategy="epoch",                 # save_strategy -> save_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    group_by_length=True,
)

# Bucketing을 적용한 Trainer
trainer_step5 = Trainer(
    model=model_bucket,
    args=training_args_step5,
    train_dataset=tokenized_nsmc["train"],
    eval_dataset=tokenized_nsmc["test"],
    compute_metrics=compute_metrics,
)

# 모델 학습 시작
start_time_step5 = time.time()
trainer_step5.train()
end_time_step5 = time.time()
training_time_step5 = end_time_step5 - start_time_step5

# 모델 성능 평가
eval_metrics_step5 = trainer_step5.evaluate()
accuracy_step5 = eval_metrics_step5['eval_accuracy']

print("\n--- Bucketing 적용 결과 ---")
print(f"훈련 시간 : {training_time_step5:.2f} 초")
print(f"평가 정확도 : {accuracy_step5:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.251800,0.247518,0.900894
2,0.179600,0.241508,0.908054
3,0.134100,0.267845,0.906334



--- Bucketing 적용 결과 ---
훈련 시간 : 9936.32 초
평가 정확도 : 0.9081


In [10]:
# ==============================================================================
# 최종 결과 비교 분석
# ==============================================================================
print("="*60)
print(" " * 20 + " 최종 결과 비교 분석")
print("="*60)
print(f"| {'항목':<20} | {'STEP 4 (일반)':<20} | {'STEP 5 (Bucketing)':<20} |")
print(f"|{'-'*22}|{'-'*22}|{'-'*22}|")
print(f"| {'평가 정확도 (Accuracy)':<20} | {accuracy_step4:<20.4f} | {accuracy_step5:<20.4f} |")
print(f"| {'총 훈련 시간 (초)':<20} | {training_time_step4:<20.2f} | {training_time_step5:<20.2f} |")
print("="*60)

                     최종 결과 비교 분석
| 항목                   | STEP 4 (일반)          | STEP 5 (Bucketing)   |
|----------------------|----------------------|----------------------|
| 평가 정확도 (Accuracy)    | 0.9068               | 0.9081               |
| 총 훈련 시간 (초)          | 9843.84              | 9936.32              |
